# アニーリングを用いたアンサンブル学習（QBoost)

[QBoost](https://arxiv.org/abs/0811.0416)は量子アニーリングを用いたアンサンブル学習の一つです。
アンサンブル学習は弱い予測器を多数用意し、その予測器の各予測結果を組み合わせることで、最終的に精度の良い予測結果を得る手法です。

## 概要と原理

このQBoostは、入力信号$\bm{x}$がどのような性質を持っているのかを精度良く識別することを目標としたアルゴリズムです。ここでは2つの値$\pm 1$のどちらを入力信号に割り当てるべきかという問題を考えましょう。例として、$\bm{x}$が画像データを表しており、その画像に写っているものが犬か猫かを識別するといったタスクを想像すると良いでしょう。アンサンブル学習では、複数の予測器を利用することで、より良い予測精度を達成すること(ブースティング)を目指します。ここでは、あまり性能の良くない予測器(弱い予測器)をたくさん用意します。性能が良くないという意味は、入力に対して正しい出力をしないことが多いことを意味します。これらの予測器の出力を$c_i (\bm{x}) \in \{ -1, 1\} \ (i=0, 1, \dots, N-1)$とします。いくつかの弱い予測器の出力の和を取ることで、より良い予測ができるというのが基本的な考え方です。これを数式で表すと

$$
C(\bm{x}) 
= \mathrm{sgn} \left( \sum_{i=0}^{N-1} w_i c_i (\bm{x}) \right) \tag{1}
$$

となります。ここで$w_i \in \{0, 1\}$で、$i$番目の予測器を使うか使わないかを表します。どの予測器を用いると、できるだけ少ない数の弱い予測器でより良い性能が得られるかを明らかにしましょう。  
このために、教師あり学習を用いて最適な$\{w_i\}$の組を求めることにします。教師データを$(\bm{x}^{(d)}, y^{(d)}) \ (d= 0, 1, \dots, D-1)$を多数用意します($D \gg 1$)。それらをできるだけ忠実に再現するように$\{w_i\}$を調整します。  
この方針をより具体的に表すと、次のハミルトニアンを$\{w_i\}$について最小化することを目指せば良いとわかります。

$$
H(\bm{w}) 
= \sum_{d=0}^{D-1} \left( \frac{1}{N} \sum_{i=0}^{N-1} w_i c_i (\bm{x}^{(d)}) - y^{(d)}\right)^2 + \lambda \sum_{i=0}^{N-1} w_i \tag{2}
$$ 

このハミルトニアンの最小化を通して、教師データ$y^{(d)}$との差ができるだけ小さくなるようにします。式(1)の右辺をそのまま使うと、符号関数があるために$w_i$の2次形式にならず、イジング模型に帰着することができません。そのため、符号関数の引数$\sum_i w_i c_i$の$1/N$倍と教師データ$y^{(d)}$との差の2乗を最小化する問題にしています。$1/N$の係数は、$\sum_i w_i c_i(\bm{x})$の最大値が$N$であるために$y^{(d)}= \pm 1$との差が大きくなりすぎないのように調整するためのものです。$\lambda (>0)$がかかった項は、あまり多くの$w_i$を1にせず、比較的少数の弱い予測器で効率良く構成するための項(正則化項)を表します。

## JijModelingによるモデル構築

### 変数の定義

式(2)で用いられている変数を、以下のようにして定義しましょう。

In [4]:
import jijmodeling as jm

# define variables
c = jm.Placeholder("c", ndim=2)
y = jm.Placeholder("y", ndim=2)
lamb = jm.Placeholder("lamb", latex="\lambda")
N = c.len_at(0, latex="N")
D = c.len_at(1, latex="D")
w = jm.BinaryVar("w", shape=(N, ))
i = jm.Element("i", belong_to=(0, N))
d = jm.Element("d", belong_to=(0, D))

`c = jm.Placeholder('c', ndim=2)`で式(2)の$c$を定義しています。
そのリストの要素数から、弱い予測器の数$N$と教師データ数$D$をそれぞれ`N, D`として定義しています。
それらを用いて、最適化に用いるバイナリ変数`w`と教師データのバイナリ値`y`を定義しています。
式(2)の$\lambda$を`lamb`として定義し、最後に式(2)で用いられている添字を`i, d`のように表しています。

### 目的関数の実装

式(2)を実装しましょう。

In [19]:
# set problem
problem = jm.Problem("QBoost")
# set objective function 1: minimize the sum of differences
problem += jm.sum(i, w[i]*c[i, d]) / N
# # set objective function 2: minimize the number of weak classifier
# problem += lamb * w[:].sum()

In [16]:
problem